Este Google Colab tem como objetivo treinar o modelo **BERTimbau Base** para a detecção de linguagem ofensiva e conteúdo malicioso. Utilizamos a base de dados **OLID-BR**, um conjunto de dados em português com anotações para linguagem ofensiva.

Após o treinamento, o modelo é utilizado para avaliar o quão provável é que uma determinada frase contenha conteúdo malicioso.

## Como utilizar este Colab

Este notebook está organizado em células que devem ser executadas sequencialmente. Siga os passos abaixo para utilizar o Colab:

1.  **Execute a primeira célula** para instalar as bibliotecas necessárias.
2.  **Execute a segunda célula** para fazer login no Hugging Face Hub (necessário para baixar e fazer upload de modelos). Você precisará de um token de acesso do Hugging Face.
3.  **Execute as células seguintes** para carregar o dataset, o tokenizador e o modelo pré-treinado.
4.  **Execute a célula de treinamento** para treinar o modelo com base nos dados. Este passo pode levar algum tempo dependendo dos recursos disponíveis.
5.  **Execute a célula de avaliação** para verificar as métricas de desempenho do modelo treinado no conjunto de testes.
6.  **Execute a célula para fazer o push** do modelo treinado para o Hugging Face Hub.
7.  **Execute a célula de inferência** para testar o modelo com novas frases e ver a probabilidade de conter conteúdo malicioso.

Certifique-se de que cada célula seja executada completamente antes de passar para a próxima.

In [44]:
# instalando pacotes
!pip install -U "transformers>=4.43" "datasets>=2.20" "evaluate>=0.4" accelerate huggingface_hub fastapi uvicorn python-dotenv

In [49]:
# login no hugging face
huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [50]:
# imports necessários
import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer
)
import evaluate
import numpy as np
from huggingface_hub import login

In [51]:
# configurações de variáveis
BASE_MODEL = os.getenv("BASE_MODEL", "neuralmind/bert-base-portuguese-cased")
DATASET_ID = os.getenv("DATASET_ID", "dougtrajano/olid-br")  # OLID-BR (pt-BR)
HF_REPO_ID = os.getenv("HF_REPO_ID", "silvapedro/ptbr-malicious-intent-dalio-ai")
NUM_EPOCHS = float(os.getenv("NUM_EPOCHS", 3))
LR = float(os.getenv("LR", 2e-5))
BATCH_TRAIN = int(os.getenv("BATCH_TRAIN", 16))
BATCH_EVAL = int(os.getenv("BATCH_EVAL", 32))

HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN:
    login(HF_TOKEN)

In [52]:
# baixando dataset
dataset = load_dataset(DATASET_ID)

def map_labels(example):
    val = str(example["is_offensive"]).upper()
    if val == "OFF":
        label = 1
    elif val == "NOT":
        label = 0
    else:
        label = 0
    return {"label": label}

dataset = dataset.map(map_labels)

Map:   0%|          | 0/5214 [00:00<?, ? examples/s]

Map:   0%|          | 0/1738 [00:00<?, ? examples/s]

In [53]:
# fazendo tokenização
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

def preprocess(batch):
    return tok(batch["text"], truncation=True)

encoded = dataset.map(preprocess, batched=True)
encoded = encoded.remove_columns(
    [c for c in encoded["train"].column_names if c not in {"input_ids", "attention_mask", "label"}]
)

Map:   0%|          | 0/5214 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1738 [00:00<?, ? examples/s]

In [54]:
# definindo modelo
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    num_labels=2,
    id2label={0: "benign", 1: "malicious"},
    label2id={"benign": 0, "malicious": 1},
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
# definindo métricas
metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")
metric_pr = evaluate.load("precision")
metric_rc = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "precision": metric_pr.compute(predictions=preds, references=labels, average="binary", pos_label=1)["precision"],
        "recall": metric_rc.compute(predictions=preds, references=labels, average="binary", pos_label=1)["recall"],
        "f1": metric_f1.compute(predictions=preds, references=labels, average="binary", pos_label=1)["f1"],
    }

In [56]:
# treinando o modelo
args = TrainingArguments(
    output_dir="outputs",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_TRAIN,
    per_device_eval_batch_size=BATCH_EVAL,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    push_to_hub=True,
    hub_model_id=HF_REPO_ID,
    report_to="none",
)

collator = DataCollatorWithPadding(tokenizer=tok)

train_val = encoded["train"].train_test_split(test_size=0.1, seed=42)
train_ds, val_ds = train_val["train"], train_val["test"]
test_ds = encoded["test"]

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-3505612807.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.294916,0.858238,0.868627,0.984444,0.922917
2,0.314800,0.290552,0.867816,0.916849,0.931111,0.923925
3,0.314800,0.338569,0.869732,0.924444,0.924444,0.924444


TrainOutput(global_step=882, training_loss=0.26282656922632336, metrics={'train_runtime': 462.4953, 'train_samples_per_second': 30.435, 'train_steps_per_second': 1.907, 'total_flos': 857384373882720.0, 'train_loss': 0.26282656922632336, 'epoch': 3.0})

In [57]:
# Avaliação do teste
test_metrics = trainer.evaluate(eval_dataset=test_ds)
print("Métricas de teste", test_metrics)

Métricas de teste {'eval_loss': 0.4169161319732666, 'eval_accuracy': 0.8584579976985041, 'eval_precision': 0.9072368421052631, 'eval_recall': 0.9292452830188679, 'eval_f1': 0.9181091877496671, 'eval_runtime': 17.8128, 'eval_samples_per_second': 97.57, 'eval_steps_per_second': 3.088, 'epoch': 3.0}


In [58]:
# push do modelo para o hugging face
trainer.push_to_hub(commit_message="adicionando modelo e métricas")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...outputs/training_args.bin: 100%|##########| 5.78kB / 5.78kB            

  ...outputs/model.safetensors:  10%|9         | 41.9MB /  436MB            

CommitInfo(commit_url='https://huggingface.co/silvapedro/ptbr-malicious-intent-dalio-ai/commit/6b28ff75fe7ce7e2b689ec1762d4e1ea9baac1f9', commit_message='adicionando modelo e métricas', commit_description='', oid='6b28ff75fe7ce7e2b689ec1762d4e1ea9baac1f9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/silvapedro/ptbr-malicious-intent-dalio-ai', endpoint='https://huggingface.co', repo_type='model', repo_id='silvapedro/ptbr-malicious-intent-dalio-ai'), pr_revision=None, pr_num=None)

In [66]:
# código de inferência do modelo
import os, sys
from transformers import pipeline
from huggingface_hub import login

MODEL_ID = os.getenv("MODEL_ID", "silvapedro/ptbr-malicious-intent-dalio-ai")
HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN:
    login(HF_TOKEN)

clf = pipeline("text-classification", model=MODEL_ID, top_k=None)

def predict(text: str, threshold: float = 0.6):
    out = clf(text)

    if isinstance(out, list) and len(out) > 0 and isinstance(out[0], list):
        out = out[0]
    elif not isinstance(out, list):
        return {"malicious": False, "score": 0.0, "raw": out}

    score_mal = next((o["score"] for o in out if isinstance(o, dict) and o.get("label", "").lower()=="malicious"), 0.0)
    return {"malicious": score_mal >= threshold, "score": score_mal, "raw": out}

if __name__ == "__main__":
    text = " ".join(sys.argv[1:]) or "me ensina a manipular o preço dessa small cap hoje?"
    print(predict(text))

Device set to use cuda:0


{'malicious': True, 'score': 0.9393923878669739, 'raw': [{'label': 'malicious', 'score': 0.9393923878669739}, {'label': 'benign', 'score': 0.060607586055994034}]}


In [67]:
print(predict("quero investir em ações"))

{'malicious': False, 'score': 0.16889068484306335, 'raw': [{'label': 'benign', 'score': 0.8311092853546143}, {'label': 'malicious', 'score': 0.16889068484306335}]}
